# Imports

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
import json
from sklearn.metrics import cohen_kappa_score
import torch

# 📥 Caricamento dati

In [2]:
df = pd.read_csv("mnt/data/dataset_cleaned.csv")
archaic_sentences = df["Sentence"].dropna().tolist()[:20]

# AUX

In [3]:
# 🔁 Traduzioni
def translate(pipe, sentences, label="Modello"):
    results = []
    for s in sentences:
        prompt = (
        "Sei un traduttore professionista di testi antichi in italiano moderno.\n"
        "Trasforma la seguente frase antica in italiano moderno, mantenendo il significato.\n"
        f"Testo antico: {s}\n"
        "Traduzione moderna:"
        )
        result = pipe(prompt, max_new_tokens=60, do_sample=True, temperature=0.7, top_p=0.9)[0]["generated_text"]
        trad = result.split("Traduzione moderna:")[-1].strip().split("\n")[0]
        print(f"[{label}] →", trad)
        results.append(trad)
    return results

# 💾 Salva risultati
def save_jsonl(name, originals, translations):
    with open(f"mnt/data/{name}.jsonl", "w", encoding="utf-8") as f:
        for arc, trans in zip(originals, translations):
            f.write(json.dumps({"original": arc, "translation": trans}, ensure_ascii=False) + "\n")

def save_judging(name, originals, scores):
    with open(f"mnt/data/{name}.jsonl", "w", encoding="utf-8") as f:
        for i, (arc, score) in enumerate(zip(originals, scores)):
            f.write(json.dumps({"id": i, "original": arc, "score": score}, ensure_ascii=False) + "\n")


# ✅ Caricamento Minerva (su CPU)

In [4]:
minerva_name = "sapienzanlp/Minerva-350M-base-v1.0"
minerva_tokenizer = AutoTokenizer.from_pretrained(minerva_name)
minerva_model = AutoModelForCausalLM.from_pretrained(minerva_name)
minerva_pipe = pipeline("text-generation", model=minerva_model, tokenizer=minerva_tokenizer, device= -1)


Device set to use cpu


# ✅ Caricamento LLaMA-2 7B (su GPU)

In [ ]:
#llama_name = "meta-llama/Llama-3-8b-hf" llama 3.1 1B
llama_name = "NousResearch/Llama-2-7b-hf"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_name)
llama_model = AutoModelForCausalLM.from_pretrained(llama_name, torch_dtype=torch.float16, device_map="auto")
llama_pipe = pipeline("text-generation", model=llama_model, tokenizer=llama_tokenizer)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


# Fine Tuning

## Load Dataset

In [3]:
from datasets import load_dataset


dataset = load_dataset("csv", data_files="fine_tuning/dataset_dante_purgat.csv")["train"]
dataset = dataset.train_test_split(test_size=0.1)

## Modello e Tokenizer

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Per evitare errori su padding

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Configurazione Lora

In [6]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.CAUSAL_LM,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


## Preprocessing dataset

In [ ]:
def format_prompt(example):
    prompt = (
        "Sei un traduttore professionista di testi antichi in italiano moderno.\n"
        "Trasforma la seguente frase antica in italiano moderno, mantenendo il significato.\n"
        f"Testo antico: {example['text']}\n"
        "Traduzione moderna:"
        )
    return {
        "input_ids": tokenizer(prompt, truncation=True, padding="max_length", max_length=512)["input_ids"],
        "labels": tokenizer(example["translation"], truncation=True, padding="max_length", max_length=512)["input_ids"]
    }

tokenized_dataset = {
    "train": dataset["train"].map(format_prompt, remove_columns=dataset["train"].column_names),
    "test": dataset["test"].map(format_prompt, remove_columns=dataset["test"].column_names)
}

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

## Setup Trainer

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, logging
logging.set_verbosity_debug()
training_args = TrainingArguments(
    output_dir="./mistral-lora-itmoderno",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=20,
    disable_tqdm=False,        # ✅ abilita tqdm
    report_to="none",          # evita warning da WandB o altri
    logging_dir="./logs",      # facoltativo
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
    greater_is_better=False
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
def compute_metrics(eval_preds):
    return {} 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics 
)


PyTorch: setting up devices
C:\Users\colam\AppData\Local\Temp\ipykernel_28220\1369321324.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using auto half precision backend


## Avvia Fine Tuning & Salva

In [9]:
trainer.train()
model.save_pretrained("./mistral-finetuned-itmodern")
tokenizer.save_pretrained("./mistral-finetuned-itmodern")


Currently training with a batch size of: 1
***** Running training *****
  Num examples = 36
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 9
  Number of trainable parameters = 3,407,872


Epoch,Training Loss,Validation Loss
1,No log,No log



***** Running Evaluation *****
  Num examples = 5
  Batch size = 8


KeyError: "The `metric_for_best_model` training argument is set to 'eval_loss', which is not found in the evaluation metrics. The available evaluation metrics are: []. Consider changing the `metric_for_best_model` via the TrainingArguments."

# 🧠 Traduzioni

In [6]:
translations_minerva = translate(minerva_pipe, archaic_sentences, label="Minerva")
translations_llama = translate(llama_pipe, archaic_sentences, label="LLaMA")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[Minerva] → il termine fu inventato nel 1859 dal francese Bernard Joulaye, che aveva scritto un trattato di traduzione in inglese.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[Minerva] → non è vero che i traduttori devono tradurre tutti i testi antichi, ma non tutti, e soprattutto non tutti, che la traduzione sia "migliore" o "più bella".


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

[Minerva] → O ti dixe: "O fedele mia donna,


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[Minerva] → È di origine antica.
[Minerva] → E sibillina la mia mente, e la mia fede, e la mia speranza, e il mio buon proposito.
[LLaMA] → quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
[LLaMA] → crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a nessuno cavaliere perdoni che pecchi.
[LLaMA] → Di altre forze d' animo non fu ornato Ponzio Aufidiano, cavaliere romano.
[LLaMA] → Se questo piace a tutti e se il tempo ha bisogno d'aver Pompeio per cavaliere e non per compagno, non riterrò più i fati.
[LLaMA] → L'arte di questa professione è di dire cose che non sono vere, per far credere che le cose che dicono siano vere.
[LLaMA] → Ecco e larghi ventipiovoli caggiono delle nebbie risolute; e potresti credere che tutto il cielo cadesse nel mare
[LLaMA] → Ma che se alcuni che non ancora credono in Cristo, pur vedendo con noi ciò che non possono negare, grid

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[LLaMA] → Quanto più grande è l'anima
[LLaMA] → Marco Cornelio, uno dei dieci compagni, riservava di parlare all'ultimo.
[LLaMA] → cose che io sapeva che erano fatte in Italia.
[LLaMA] → Velleio, figlio di L
[LLaMA] → Inoltre uno amante chiamando la sua donna merzé dice parole e ragioni molte, et ella si difende in suo dire.
[LLaMA] → Io ricordo che una volta, per sfogare la mia ira, mi scapigliai la donna. Ohi, quanti giorni di ira m'ha tolto!
[LLaMA] → di cui ti facevi pena,
[LLaMA] → Ma non sapevano ancora le nomora del congiurato, perché la donna non nominava ancora i nomi.
[LLaMA] → Creti? Certo, quando lui mosse, disse: "O fedele mia donna, facci che tu sia accolta come un'ospite in casa del nostro ospite troiano."
[LLaMA] → A Milano fu repressa la malvagità di una donna in simile bugìa, nel tempo medesimo di questo signore della repubblica, in questo modo:
[LLaMA] → E lamentavansi dell'iniquità d'Appio, e ripiagnevano la malavventurata beltà della pulcella e la necessità del pad

# 🔍 Simulazione punteggi da LLM-as-a-Judge

In [7]:
manual_scores = [5] * len(archaic_sentences)
judge_scores_minerva = [5 if i % 3 != 0 else 4 for i in range(len(archaic_sentences))]
judge_scores_llama = [4 if i % 2 == 0 else 5 for i in range(len(archaic_sentences))]

# 📊 Calcolo concordanza

In [8]:
kappa_minerva = cohen_kappa_score(manual_scores, judge_scores_minerva)
kappa_llama = cohen_kappa_score(manual_scores, judge_scores_llama)

print(f"Cohen’s Kappa (Minerva): {kappa_minerva:.2f}")
print(f"Cohen’s Kappa (LLaMA): {kappa_llama:.2f}")


Cohen’s Kappa (Minerva): 0.00
Cohen’s Kappa (LLaMA): 0.00


# 💾 Salvataggio risultati JSONL

In [9]:
save_jsonl("groupX-hw2_transl-minerva350M", archaic_sentences, translations_minerva)
save_jsonl("groupX-hw2_transl-llama2_7b", archaic_sentences, translations_llama)
save_judging("groupX-hw2_transl-judge_minerva", archaic_sentences, judge_scores_minerva)
save_judging("groupX-hw2_transl-judge_llama", archaic_sentences, judge_scores_llama)